In [195]:
from IPython.display import HTML

HTML('''
<script>
  function code_toggle() {    if (code_shown){      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {$('div.input').show('500');$('#toggleButton').val('Hide Code')    }code_shown = !code_shown
  }  $( document ).ready(function(){code_shown=false;$('div.input').hide()
  });</script><form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')

In [198]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import re

import os
import json

In [207]:
def strip_exp_names(e, iden='FINAL_RESULTS'):
    i = e.find(iden)+len(iden)+1
    return e[i:-5]

In [210]:
model_path = '../attack_results/white_box/'
all_nets =  list(os.listdir(model_path))
all_nets = [a for a in all_nets if 'FINAL_RESULTS' in a]
# all_nets = [strip_exp_names(a) for a in all_nets]
all_nets_cifar10 = [a for a in all_nets if 'cifar100' not in a]

In [211]:
nets = all_nets_cifar10
attack_results = {}

for net in nets:
    fname = model_path+net
    with open(fname,'r+') as json_data:
        attack_results[strip_exp_names(net)] = list(json.load(json_data).values())
        
df = pd.DataFrame.from_dict(attack_results, columns=['Clean', 
                                                     'FGSM 0.03125', 'PGD 0.03125',
                                                     'FGSM 0.0625', 'PGD 0.0625',
                                                    ], orient='index')

for row in df.iterrows():
    s = row[0].split('_')
#     print(s)
    if s[0] == 'transfer':
#         df.loc[row[0], 'Net'] = s[3]
        df.loc[row[0], 'Type'] = 'Transfer'
        df.loc[row[0], 'Source Adversary'] = s[4]
        df.loc[row[0], 'Target Adversary'] = s[5]
        df.loc[row[0], 'Transfer'] = s[1] + ' ' + s[2]
    else:
#         df.loc[row[0], 'Net'] = s[0]
        df.loc[row[0], 'Type'] = 'Baseline'
        df.loc[row[0], 'Source Adversary'] = s[2] if len(s)==3 else 'nat'
        df.loc[row[0], 'Target Adversary'] = ''
        df.loc[row[0], 'Transfer'] = 'No transfer'
        
df[df.select_dtypes(include=['number']).columns] *= 100
df=np.round(df,1)

In [212]:
idx = ['Type', 'Transfer', 'Source Adversary', 'Target Adversary']
df = df.sort_values(idx)
df.set_index(idx)#[['Clean', 'FGSM 0.0625','PGD 0.0625']]
# df

Clean  \
Type     Transfer       Source Adversary Target Adversary          
Baseline No transfer    fgsm                                87.1   
                        nat                                 92.5   
                        pgd                                 83.9   
Transfer 12 layers      fgsm             fgsm               87.2   
                                         nat                89.1   
                        nat              fgsm               77.3   
                                         nat                91.2   
                                         pgd                60.8   
                        pgd              pgd                81.3   
         all layers     fgsm             fgsm               85.7   
                                         nat                91.9   
                        nat              fgsm               88.1   
                                         nat                92.6   
                                         pgd                81.5   
                        pgd              pgd                82.6   
         feat extractor fgsm             fgsm               66.6   
                                         nat                69.0   
                        nat              fgsm               69.3   
                                         nat                73.1   
                                         pgd                52.2   
                        pgd              pgd                61.3   

                                                           FGSM 0.03125  \
Type     Transfer       Source Adversary Target Adversary                 
Baseline No transfer    fgsm                                       84.4   
                        nat                                        25.2   
                        pgd                                        62.8   
Transfer 12 layers      fgsm             fgsm                      81.6   
                                         nat                       55.3   
                        nat              fgsm                      44.1   
                                         nat                       22.5   
                                         pgd                       22.7   
                        pgd              pgd                       62.6   
         all layers     fgsm             fgsm                      78.2   
                                         nat                       27.9   
                        nat              fgsm                      81.5   
                                         nat                       23.6   
                                         pgd                       63.8   
                        pgd              pgd                       65.1   
         feat extractor fgsm             fgsm                      58.8   
                                         nat                       33.2   
                        nat              fgsm                      11.5   
                                         nat                        6.0   
                                         pgd                       13.4   
                        pgd              pgd                       40.8   

                                                           PGD 0.03125  \
Type     Transfer       Source Adversary Target Adversary                
Baseline No transfer    fgsm                                      20.0   
                        nat                                        0.1   
                        pgd                                       59.3   
Transfer 12 layers      fgsm             fgsm                      6.7   
                                         nat                       7.4   
                        nat              fgsm                      4.7   
                                         nat                       0.0   
                                         pgd                       4.1   
                        pgd 

# Black box attacks

In [74]:
js = json.load(open("../experiments_results/attack_results/black_box_results_all_layers.json"))